# Nulls in Spark

In [1]:
import pandas as pd
import pyspark.sql.functions as F 

In [2]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder
    .master("local[*]")
    .appName("Nulls in Spark")
    .getOrCreate()
        )

## Nulls and data types
Besides the null, which corresponds to a python None,  there're also defacto nulls like NaN and ''.
Note the NaN is a float or double data type. It cannot be part of an integer column nor a string column in spark. 
And '' can only show up in a string column.

In [36]:
df = spark.createDataFrame([(1., None, 'Tom'), (None, 2, None), (float('nan'), 2, '')], ("a", "b", "c"))

In [29]:
df.show()
df

+----+----+----+
|   a|   b|   c|
+----+----+----+
| 1.0|null| Tom|
|null|   2|null|
| NaN|   2|    |
+----+----+----+



DataFrame[a: double, b: bigint, c: string]

## Detection of nulls
F.isnull and df.column.isNull() picks up nulls but not NaN nor ''

In [34]:
expr = [F.isnull(column) for column in df.columns]
df.select(*expr).show()
df.select(df.a.isNull(), df.b.isNull(),df.c.isNull()).show()

+-----------+-----------+-----------+
|(a IS NULL)|(b IS NULL)|(c IS NULL)|
+-----------+-----------+-----------+
|      false|       true|      false|
|       true|      false|       true|
|      false|      false|      false|
+-----------+-----------+-----------+

+-----------+-----------+-----------+
|(a IS NULL)|(b IS NULL)|(c IS NULL)|
+-----------+-----------+-----------+
|      false|       true|      false|
|       true|      false|       true|
|      false|      false|      false|
+-----------+-----------+-----------+



F.isnan() picks up NaN's only, and a null will not trigger an error. 

In [37]:
expr = [F.isnan(column) for column in df.columns]
df.select(*expr).show()

+--------+--------+--------+
|isnan(a)|isnan(b)|isnan(c)|
+--------+--------+--------+
|   false|   false|   false|
|   false|   false|   false|
|    true|   false|   false|
+--------+--------+--------+



F.col(column)=='' picks up '''s only, a null does not trigger an error, a type mismatch triggers a null. 

In [38]:
expr = [F.col(column)=='' for column in df.columns]
df.select(*expr).show()

+------+------+------+
|(a = )|(b = )|(c = )|
+------+------+------+
|  null|  null| false|
|  null|  null|  null|
|  null|  null|  true|
+------+------+------+



## Counts with nulls
F.count() does not count nulls but counts NaNs and ''s

In [39]:
expr = [F.count(column) for column in df.columns]
df.select(*expr).show()

+--------+--------+--------+
|count(a)|count(b)|count(c)|
+--------+--------+--------+
|       2|       2|       2|
+--------+--------+--------+



Therefore needs to subtract NaN and '' explicitly to get the true null rate. Note F.col(column)=='' triggers null in non-string columns and needs to be treated separately. 

In [58]:
string_columns = [column for (column, type) in df.dtypes if type=='string']
non_string_columns = [column for (column, type) in df.dtypes if type!='string']
expr1 = [F.count(column)-F.sum(F.isnan(column).cast('int')) for column in non_string_columns]
expr2 = [F.count(column)-F.sum((F.col(column)=='').cast('int')) for column in string_columns]
df.select(*expr1+expr2).show()

+---------------------------------------+---------------------------------------+-------------------------------------+
|(count(a) - sum(CAST(isnan(a) AS INT)))|(count(b) - sum(CAST(isnan(b) AS INT)))|(count(c) - sum(CAST((c = ) AS INT)))|
+---------------------------------------+---------------------------------------+-------------------------------------+
|                                      1|                                      2|                                    1|
+---------------------------------------+---------------------------------------+-------------------------------------+



Note for column b, the adjustment is not needed because as an integer column, it cannot contain NaN nor ''